In [1]:
import pandas as pd
import os

In [2]:
%pwd

'/config/workspace/Wine-Quality-ML-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/config/workspace/Wine-Quality-ML-Project'

In [5]:

from dataclasses import dataclass
from pathlib import Path

# entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from Wine_Quality_ML_Project.constants import *
from Wine_Quality_ML_Project.utils.common import read_yaml,create_directories 

In [7]:
class ConfigurationManager:
    
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )


        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from Wine_Quality_ML_Project import logger
from Wine_Quality_ML_Project.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_files(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_files()
except Exception as e:
    raise e

[2024-01-18 10:46:54,109:INFO:common:yaml file: config/config.yaml loaded]
[2024-01-18 10:46:54,110:INFO:common:yaml file: params.yaml loaded]
[2024-01-18 10:46:54,114:INFO:common:yaml file: schema.yaml loaded]
[2024-01-18 10:46:54,116:INFO:common:created directory at: artifacts]
[2024-01-18 10:46:54,117:INFO:common:created directory at: artifacts/data_ingestion]
[2024-01-18 10:46:54,851:INFO:601752412:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3a0d88cacc77beac7967671b263cb53d7852b9f13bd89c20dedebb8aaf1c9965"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BB8E:84801:91F105:A04B95:65A8B446
Accept-Ranges: bytes
Date: Thu, 18 Jan 2024 05:16:54 GMT
Via: 1.1 varnish
X-Served-By: cache-